In [1]:
import sys
print(sys.executable)
!pwd

/rwthfs/rz/cluster/home/pe432442/MRP2/venv/bin/python3
/rwthfs/rz/cluster/home/pe432442/MRP2/kinetics-blstm/notebooks


In [2]:
import sys
sys.path.append('..')

import tensorflow as tf
from tensorflow.keras import layers

In [3]:
from utils import PatientLoader
from processing.signal import *
import constants as const

In [4]:
!nvidia-smi

Mon Jun 22 17:32:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 440.44       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-SXM2...  Off  | 00000000:06:00.0 Off |                    0 |
| N/A   38C    P0    43W / 300W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-SXM2...  Off  | 00000000:84:00.0 Off |                    0 |
| N/A   42C    P0    38W / 300W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [5]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [5]:
pl = PatientLoader(1, data_dir="../data")
print(pl.get_features("metrics").shape, pl.get_audio().shape)


(6, 30230) (14511116,)


In [6]:
14511116 / 30230

480.0236850810453

In [76]:
import constants as c
from pathlib import Path
import librosa
from IPython.display import Audio
from processing.speech import extract_world_parameters
# from processing.speech import world_reconstruct_audio
from pyworld import wav2world
from numpy import ascontiguousarray as ctg_array
import pyworld as pw
from scipy.io.wavfile import read
from scipy.signal import decimate

audios = Path("."+c.BALDEY_AUDIO_DIR).rglob("*.wav")
a1, r = librosa.load(next(audios), sr=None)
# r, a1 = read(next(audios))
# a1 = a1 / 32767
# a1 = np.r_[np.zeros_like(a1), a1, np.zeros_like(a1)]
target_fs = 16000

# downsampled = librosa.resample(a1, orig_sr=r, target_sr=target_fs)


# Audio(downsampled, rate=r)

pars = extract_world_parameters(a1, fs=r, frame_period=1,
                                return_individual=True)
# pars['encoded_ap'] = pars['encoded_ap'].reshape()
rec = world_reconstruct_audio(fs=target_fs, frame_period=1, **pars)

a1.shape, rec.shape

Audio(rec, rate=target_fs)

In [47]:
pl.get_file_format()

'../data/kh1_1_{}.npy'

### Eduardo's model

In [10]:
from sklearn.model_selection import train_test_split

def split_with_overlap(data,window_size, window_overlap):
    length = data.shape[0]
    result = []
    for i in range(0,length,window_size-window_overlap):
        if length >= i + window_size:
            result.append(data[i:i+window_size,:])
    return result



batch_size = 1000
overlap_size = 200

print('Loading data...')

rawseeg = pl.get_seeg()
data = detrend(rawseeg)
hg = high_gamma(data, pl.seeg_fs)
X_high_gamma = analytic_amp(hg)
lp = low_component(rawseeg, pl.seeg_fs)
X_low_component = analytic_amp(lp)


batch_size = 1000
overlap_size = 200

print('Loading data...')

Y = pl.get_features("metrics")
Y = Y.T
X = np.hstack((X_high_gamma, X_low_component))
X = X[:Y.shape[0], :]


Loading data...
Setting up band-pass filter from 70 - 1.7e+02 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 70.00, 170.00 Hz: -6.02, -6.02 dB

Setting up band-stop filter from 98 - 1e+02 Hz

IIR filter parameters
---------------------
Butterworth bandstop zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 98.00, 102.00 Hz: -6.02, -6.02 dB

Setting up band-stop filter from 1.5e+02 - 1.5e+02 Hz

IIR filter parameters
---------------------
Butterworth bandstop zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 148.00, 152.00 Hz: -6.02, -6.02 dB

Setting up low-pass filter at 30 Hz

IIR filter parameters
---------------------
Butterworth lowpass zero-phase (two-pass forward and reverse) non

In [14]:
rawseeg.shape, data.shape, hg.shape, X.shape, Y.shape

((309569, 107), (309569, 107), (309569, 107), (30230, 214), (30230, 6))

In [ ]:

num_features = X.shape[1]

X = split_with_overlap(X,batch_size,overlap_size)
Y = split_with_overlap(Y,batch_size,overlap_size)

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

y_train = np.array(y_train)
y_test = np.array(y_test)
x_train = np.array(x_train)
x_test = np.array(x_test)

print(f'X train shape: {x_train.shape}')
print(f'Y train shape: {y_train.shape}')
print(f'X test shape: {x_test.shape}')
print(f'Y test shape: {y_test.shape}')

Y = np.load('./data/kh4_1_metrics.npy')
Y = Y.T
X = np.hstack((X_high_gamma, X_low_component))
X = X[:Y.shape[0], :]

num_features = X.shape[1]

X = split_with_overlap(X,batch_size,overlap_size)
Y = split_with_overlap(Y,batch_size,overlap_size)

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

y_train = np.array(y_train)
y_test = np.array(y_test)
x_train = np.array(x_train)
x_test = np.array(x_test)

print(f'X train shape: {x_train.shape}')
print(f'Y train shape: {y_train.shape}')
print(f'X test shape: {x_test.shape}')
print(f'Y test shape: {y_test.shape}')

model = Sequential()
model.add(Dense(6,input_shape=(batch_size,num_features)))
model.add(Bidirectional(LSTM(3,return_sequences=True)))
model.add(Dense(6))
model.add(Dropout(0.1))
model.add(Activation('softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy',tf.keras.metrics.MeanSquaredError()])

#model.fit(x_train, y_train,
#          epochs=100,
#          validation_data=(x_test, y_test))


print(model.summary())


### Anton's model

In [6]:
def decoder_model(in_shape):
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=in_shape))
    model.add(layers.Dropout(0.25))
    model.add(layers.Bidirectional(layers.LSTM(100, return_sequences=True)))
    model.add(layers.Dropout(0.25))
    model.add(layers.Bidirectional(layers.LSTM(100, return_sequences=True)))
    model.add(layers.Dropout(0.25))
    model.add(layers.Bidirectional(layers.LSTM(100)))
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(1))
    opt = tf.keras.optimizers.Adam(learning_rate=0.001,
                                   beta_1=0.9,
                                   beta_2=0.999,
                                   epsilon=1e-08)
    model.compile(optimizer=opt, loss="mean_squared_error")
    return model

#### Fit on patient 4

In [11]:
loader = PatientLoader(4, data_dir='../data')
art_features = loader.get_features('metrics_1k')
gen_art_features = loader.get_features('gen_kinetics')
world_10_params = loader.get_features(f'{const.WORLD_PARAM_NAME}_10')
world_1_params = loader.get_features(f'{const.WORLD_PARAM_NAME}_1')

In [12]:
art_features.shape, \
gen_art_features.shape,\
world_10_params.shape, \
world_1_params.shape, \
loader.get_seeg().shape \

((6, 300300), (5561, 6), (30032, 516), (300315, 516), (307519, 133))

In [19]:
sequence_length = 60
batch_size = 100
shuffle_size = 10000

In [20]:
def make_dataset(art_features,
                 world_features,
                 sequence_length,
                 batch_size,
                 shuffle_size=10000,
                 step_size=1):
    # Get the features to the same length
    art_features = art_features.T
    n = min(art_features.shape[0], world_features.shape[0])
    art_features = art_features[:n, ...]
    world_features = world_features[:n, ...]
    X = []
    y = []
    for i in range(0, n - sequence_length, step_size):
        X.append(art_features[i:(i + sequence_length), ...])
        y.append(world_features[i + (sequence_length // 2)])
    
    dset = tf.data.Dataset.from_tensor_slices((X, y))
    dset = dset.shuffle(shuffle_size).batch(batch_size)
    return dset

dataset = make_dataset(art_features,
                       world_1_params,
                       sequence_length=sequence_length,
                       batch_size=batch_size,
                       shuffle_size=shuffle_size)

In [35]:
art_features.shape

(6, 300300)

In [36]:
input_shape = (sequence_length, art_features.T.shape[1]) # (seq_length, 6)
model = decoder_model(input_shape)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_8 (Dropout)          (None, 60, 6)             0         
_________________________________________________________________
bidirectional_6 (Bidirection (None, 60, 200)           85600     
_________________________________________________________________
dropout_9 (Dropout)          (None, 60, 200)           0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 60, 200)           240800    
_________________________________________________________________
dropout_10 (Dropout)         (None, 60, 200)           0         
_________________________________________________________________
bidirectional_8 (Bidirection (None, 200)               240800    
_________________________________________________________________
dropout_11 (Dropout)         (None, 200)              

In [39]:
model.fit(dataset, epochs=10)

Epoch 1/10
3003/3003 [==============================] - 64s 21ms/step - loss: 16.6572
Epoch 2/10
3003/3003 [==============================] - 64s 21ms/step - loss: 16.6569
Epoch 3/10
3003/3003 [==============================] - 64s 21ms/step - loss: 16.6568
Epoch 4/10
3003/3003 [==============================] - 64s 21ms/step - loss: 16.6564
Epoch 5/10
3003/3003 [==============================] - 64s 21ms/step - loss: 16.6566
Epoch 6/10
3003/3003 [==============================] - 64s 21ms/step - loss: 16.6560
Epoch 7/10
3003/3003 [==============================] - 64s 21ms/step - loss: 16.6558
Epoch 8/10
3003/3003 [==============================] - 64s 21ms/step - loss: 16.6557
Epoch 9/10
3003/3003 [==============================] - 64s 21ms/step - loss: 16.6556
Epoch 10/10
3003/3003 [==============================] - 64s 21ms/step - loss: 16.6554


In [13]:
list(range(10))[np.random.permutation(10).tolist()]

TypeError: list indices must be integers or slices, not list

In [25]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [27]:
!squeue -u pe432442

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          14655040        ih mrp-jupy pe432442  R    2:35:22      1 lnih173
